House Price Prediction Model Training and Logging in ML FLow
-

In [1]:
!pip install mlflow


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


run this on cmd first-

mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns --host 127.0.0.1 --port 5000


In [3]:
import mlflow
import mlflow.sklearn

print("MLflow version:", mlflow.__version__)

# 1. Set the URI FIRST so MLflow knows WHERE to look/create
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# 2. Set the experiment (This creates it on the server if it's missing)
experiment_name = "House_Price_Prediction_Model_Train_Log"
mlflow.set_experiment(experiment_name)

2026/01/28 09:37:30 INFO mlflow.tracking.fluent: Experiment with name 'House_Price_Prediction_Model_Train_Log' does not exist. Creating a new experiment.


MLflow version: 3.8.1


<Experiment: artifact_location='file:C:/Users/RUCHITA DIGHE/Desktop/DBDA_Housing_Project/mlruns/5', creation_time=1769573250669, experiment_id='5', last_update_time=1769573250669, lifecycle_stage='active', name='House_Price_Prediction_Model_Train_Log', tags={}>

In [4]:
# 3. Double check the ID (Optional but helpful for debugging)
exp = mlflow.get_experiment_by_name(experiment_name)
print(f"Experiment ID: {exp.experiment_id}")
print(f"Active Experiment: {mlflow.get_experiment_by_name(experiment_name)}")

Experiment ID: 5
Active Experiment: <Experiment: artifact_location='file:C:/Users/RUCHITA DIGHE/Desktop/DBDA_Housing_Project/mlruns/5', creation_time=1769573250669, experiment_id='5', last_update_time=1769573250669, lifecycle_stage='active', name='House_Price_Prediction_Model_Train_Log', tags={}>


Model Training for House Price Prediction of USA
-

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install xgboost
!pip install lightgbm
!pip install catboost


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import os

Load file
-

In [25]:
df=pd.read_csv("clean_houseprice_data_1.csv")
df.head()

,livingArea in sq.ft,beds,baths,address,zipcode,city,state,country,price in $
0,3563.491860,8,7,"7658 W Pocahontas Ave, Tampa, FL 33615, Anchor...",99516,Anchorage,AK,USA,1.003360e+06
1,560.000000,1,1,"8057 Reynolds St, Staten Island, NY 10305, Ken...",99611,Kenai,AK,USA,1.404153e+05
2,2390.959601,5,5,"6796 W Comanche Ave, Tampa, FL 33603, Nenana, ...",99760,Nenana,AK,USA,5.281063e+05
3,560.000000,1,1,"1159 Saleta St, Pensacola, FL 32534, North Pol...",99705,North Pole,AK,USA,1.454507e+05
4,560.000000,1,1,"6721 Prospect St NW, Washington, DC 20007, Pal...",99645,Palmer,AK,USA,1.433320e+05


In [26]:
df['price in $']=df['price in $'].astype('float32')
df['price in $']=df['price in $'].astype('int64')
df.head()

,livingArea in sq.ft,beds,baths,address,zipcode,city,state,country,price in $
0,3563.491860,8,7,"7658 W Pocahontas Ave, Tampa, FL 33615, Anchor...",99516,Anchorage,AK,USA,1003360
1,560.000000,1,1,"8057 Reynolds St, Staten Island, NY 10305, Ken...",99611,Kenai,AK,USA,140415
2,2390.959601,5,5,"6796 W Comanche Ave, Tampa, FL 33603, Nenana, ...",99760,Nenana,AK,USA,528106
3,560.000000,1,1,"1159 Saleta St, Pensacola, FL 32534, North Pol...",99705,North Pole,AK,USA,145450
4,560.000000,1,1,"6721 Prospect St NW, Washington, DC 20007, Pal...",99645,Palmer,AK,USA,143332


In [27]:
df = df.drop(columns=['country','address','zipcode'])
df.columns

Index(['livingArea in sq.ft', 'beds', 'baths', 'city', 'state', 'price in $'], dtype='object')

Define features & target
-

In [28]:
X = df.drop(columns=['price in $'])
y = df['price in $']

num_cols = ['livingArea in sq.ft', 'beds', 'baths']
cat_cols = ['city', 'state']

In [29]:
X

,livingArea in sq.ft,beds,baths,city,state
0,3563.491860,8,7,Anchorage,AK
1,560.000000,1,1,Kenai,AK
2,2390.959601,5,5,Nenana,AK
3,560.000000,1,1,North Pole,AK
4,560.000000,1,1,Palmer,AK
...,...,...,...,...,...
179995,1822.764158,4,4,Henrico,VA
179996,1597.964758,3,3,Richmond,VA
179997,4445.693591,10,10,Henrico,VA
179998,5586.193955,10,9,Henrico,VA


Train test split
-

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Preprocessor (Scaler + Encoder)
-

In [31]:
# One-hot encoder 
from sklearn.compose import ColumnTransformer


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols)
    ]
)

In [34]:
def total_combinations(*lists):
    total = 1
    for l in lists:
        total *= len(l)
    return total

Helper function (reuse for all models):
-

In [32]:
import pickle

def evaluate_and_log(pipeline, model_name, params):
    """Evaluate on test set, log metrics & model to MLflow, save pickle"""
    y_pred = pipeline.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Save pickle locally
    model_filename = f"{model_name}.pkl"
    with open(model_filename, "wb") as f:
        pickle.dump(pipeline, f)

    # Log to MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.set_tag("model_type", model_name)

        # Log metrics
        mlflow.log_metric("R2", r2)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MAE", mae)

        # Log hyperparameters
        for k, v in params.items():
            mlflow.log_param(k, v)

        # Log full pipeline
        mlflow.sklearn.log_model(pipeline, artifact_path=model_name)

        # Log pickle as artifact
        mlflow.log_artifact(model_filename)

    print(f"{model_name} logged + pickle saved | R2={r2:.4f}")

Linear Regression Model
-

In [33]:
from sklearn.pipeline import Pipeline


lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

lr_pipeline.fit(x_train, y_train)
evaluate_and_log(lr_pipeline, "LinearRegression", {})

2026/01/28 09:59:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run LinearRegression at: http://127.0.0.1:5000/#/experiments/5/runs/c099ee5b66c942519783de47a0a19fad
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
LinearRegression logged + pickle saved | R2=0.6788


XGBoost Model with hyperparameter tuning
-

In [35]:
trees = [50, 100, 150, 200]
rates = [0.001, 0.01, 0.1, 0.2, 0.5, 0.8]
depths = [2, 3, 4, 5] 

total = total_combinations(trees, rates, depths)
best_score = -1
best_pipeline = None
best_params = {}

with tqdm(total=total, desc="XGBoost tuning") as pbar:
    for t in trees:
        for d in depths:
            for r in rates:
                model = XGBRegressor(
                    random_state=42,
                    n_estimators=t,
                    max_depth=d,
                    learning_rate=r
                )
                pipeline = Pipeline([
                    ('preprocessor', preprocessor),
                    ('model', model)
                ])
                pipeline.fit(x_train, y_train)
                r2 = r2_score(y_test, pipeline.predict(x_test))
                if r2 > best_score:
                    best_score = r2
                    best_pipeline = pipeline
                    best_params = {"n_estimators": t, "max_depth": d, "learning_rate": r}
                pbar.update(1)

evaluate_and_log(best_pipeline, "XGBoost", best_params)

XGBoost tuning: 100%|██████████| 96/96 [00:29<00:00,  3.29it/s]
2026/01/28 10:01:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/5/runs/4e4df5e5a4ea486a9117ee9b4c4aaff1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
XGBoost logged + pickle saved | R2=0.9536


Decision Tree hyperparameter Tuning
-

In [36]:
depths =[None,3,4,5,6,7]
min_samples = [2,10,0.025,0.01,0.05,0.1]
min_leaf = [1,10,0.025,0.01,0.05,0.1]

total = total_combinations(depths, min_samples, min_leaf)

best_score = -1
best_pipeline = None
best_params = {}

with tqdm(total=total, desc="DecisionTree tuning") as pbar:
    for d in depths:
        for s in min_samples:
            for l in min_leaf:
                model = DecisionTreeRegressor(
                    random_state=42,
                    max_depth=d,
                    min_samples_split=s,
                    min_samples_leaf=l
                )

                pipeline = Pipeline([
                    ('preprocessor', preprocessor),
                    ('model', model)
                ])

                pipeline.fit(x_train, y_train)
                r2 = r2_score(y_test, pipeline.predict(x_test))

                if r2 > best_score:
                    best_score = r2
                    best_pipeline = pipeline
                    best_params = {
                        "max_depth": d,
                        "min_samples_split": s,
                        "min_samples_leaf": l
                    }

                pbar.update(1)

evaluate_and_log(best_pipeline, "DecisionTree", best_params)

DecisionTree tuning: 100%|██████████| 216/216 [01:01<00:00,  3.50it/s]
2026/01/28 10:04:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run DecisionTree at: http://127.0.0.1:5000/#/experiments/5/runs/bbd5e8ad242b43a3b766e305b21ec5e1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
DecisionTree logged + pickle saved | R2=0.9915


Random Forest hyperparameter Tuning
-

In [37]:
features =[None, 'sqrt', 'log2']
n_est = [25, 50,100,150, 200]

total = total_combinations(features, n_est)

best_score = -1
best_pipeline = None
best_params = {}

with tqdm(total=total, desc="RandomForest tuning") as pbar:
    for f in features:
        for n in n_est:
            model = RandomForestRegressor(
                random_state=42,
                max_features=f,
                n_estimators=n,
                n_jobs=-1
            )

            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('model', model)
            ])

            pipeline.fit(x_train, y_train)
            r2 = r2_score(y_test, pipeline.predict(x_test))

            if r2 > best_score:
                best_score = r2
                best_pipeline = pipeline
                best_params = {
                    "max_features": f,
                    "n_estimators": n
                }

            pbar.update(1)

evaluate_and_log(best_pipeline, "RandomForest", best_params)

RandomForest tuning: 100%|██████████| 15/15 [53:15<00:00, 213.00s/it]
2026/01/28 10:57:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run RandomForest at: http://127.0.0.1:5000/#/experiments/5/runs/d81db08cf0564937b2d8774e7094fa1b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
RandomForest logged + pickle saved | R2=0.9938


Light GBM  model with hyperparameter tuning
-

In [38]:
from lightgbm import LGBMRegressor
trees= [50, 100, 150, 200]
rates = [0.01, 0.1, 0.2, 0.001]
depths=[2, 3, 4, 5,7]

total = total_combinations(trees, depths, rates)

best_score = -1
best_pipeline = None
best_params = {}

with tqdm(total=total, desc="LightGBM tuning") as pbar:
    for t in trees:
        for r in rates:
            for d in depths:
                model = LGBMRegressor(
                    random_state=42,
                    n_estimators=t,
                    learning_rate=r,
                    max_depth=d,
                    verbose=-1
                )

                pipeline = Pipeline([
                    ('preprocessor', preprocessor),
                    ('model', model)
                ])

                pipeline.fit(x_train, y_train)
                r2 = r2_score(y_test, pipeline.predict(x_test))

                if r2 > best_score:
                    best_score = r2
                    best_pipeline = pipeline
                    best_params = {
                        "n_estimators": t,
                        "learning_rate": r,
                        "max_depth": d
                    }

                pbar.update(1)

evaluate_and_log(best_pipeline, "LightGBM", best_params)

LightGBM tuning:   0%|          | 0/80 [00:00<?, ?it/s]c:\Users\RUCHITA DIGHE\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
LightGBM tuning:   1%|▏         | 1/80 [00:02<03:44,  2.85s/it]c:\Users\RUCHITA DIGHE\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
LightGBM tuning:   2%|▎         | 2/80 [00:03<01:44,  1.34s/it]c:\Users\RUCHITA DIGHE\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
LightGBM tuning:   4%|▍         | 3/80 [00:03<01:06,  1.17it/s]c:\Users\RUCHITA DIGHE\AppData\Local\Programs\Python\Python313\Lib

🏃 View run LightGBM at: http://127.0.0.1:5000/#/experiments/5/runs/92268ce7b392434fa23bb68c9f90b6b8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
LightGBM logged + pickle saved | R2=0.9511


Cat Boosting Regressor
-

In [40]:
from catboost import CatBoostRegressor

# Identify categorical column indices
cat_features = [X.columns.get_loc(col) for col in cat_cols]

trees = [50, 100, 150, 200]
rates = [0.01, 0.1, 0.2, 0.001]
depths = [2, 3, 4, 5, 7]

total = total_combinations(trees, rates, depths)

best_score = -1
best_model = None
best_params = {}

with tqdm(total=total, desc="CatBoost tuning") as pbar:
    for t in trees:
        for r in rates:
            for d in depths:
                model = CatBoostRegressor(
                    random_state=42,
                    n_estimators=t,
                    learning_rate=r,
                    max_depth=d,
                    verbose=False
                )

                model.fit(
                    x_train,
                    y_train,
                    cat_features=cat_features
                )

                r2 = r2_score(y_test, model.predict(x_test))

                if r2 > best_score:
                    best_score = r2
                    best_model = model
                    best_params = {
                        "n_estimators": t,
                        "learning_rate": r,
                        "max_depth": d
                    }

                pbar.update(1)

CatBoost tuning: 100%|██████████| 80/80 [09:37<00:00,  7.22s/it]


Log to ml flow separately

In [41]:
with mlflow.start_run(run_name="CatBoost"):
    mlflow.set_tag("model_type", "CatBoost")

    mlflow.log_metric("R2", best_score)
    mlflow.log_params(best_params)

    mlflow.catboost.log_model(best_model, artifact_path="CatBoost")

2026/01/28 11:10:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run CatBoost at: http://127.0.0.1:5000/#/experiments/5/runs/3c83a6820e5940cd986084c6fd78f32c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


Elastic Net
-

In [42]:
from sklearn.linear_model import ElasticNet

# Define parameter grids
alphas = [0.01, 0.1, 1, 10, 100]          # regularization strength
l1_ratios = [0.2, 0.5, 0.8]               # mix of L1 (Lasso) and L2 (Ridge)

total = total_combinations(alphas, l1_ratios)

best_score = -1
best_pipeline = None
best_params = {}

with tqdm(total=total, desc="ElasticNet tuning") as pbar:
    for a in alphas:
        for l1 in l1_ratios:
            model = ElasticNet(
                alpha=a,
                l1_ratio=l1,
                max_iter=10000,
                random_state=42
            )

            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('model', model)
            ])

            pipeline.fit(x_train, y_train)
            r2 = r2_score(y_test, pipeline.predict(x_test))

            if r2 > best_score:
                best_score = r2
                best_pipeline = pipeline
                best_params = {
                    "alpha": a,
                    "l1_ratio": l1
                }

            pbar.update(1)

evaluate_and_log(best_pipeline, "ElasticNet", best_params)

ElasticNet tuning:   0%|          | 0/15 [00:00<?, ?it/s]

ElasticNet tuning: 100%|██████████| 15/15 [01:19<00:00,  5.31s/it]
2026/01/28 11:11:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run ElasticNet at: http://127.0.0.1:5000/#/experiments/5/runs/ab3f73a237d5472e87cfac4de45006b0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
ElasticNet logged + pickle saved | R2=0.6746
